In [1]:
# This script reads in the list of forest inventory plots from the OSU GNN dataset
# We want to summarize each Forest Condition ID into 2" diameter classes and species
# We will query this summary DataFrame to calculate the volume (board-foot, cubic foot, and tonnage)
# of potential wood products on-site given a diameter range and list of species provided by the user

In [2]:
# load the packages
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
from matplotlib import pyplot as plt
import math
from ARBCARVolumeEquations import *

In [3]:
workdir = 'C:\Users\ddiaz\Desktop'

In [ ]:
CSV = 'GNN_TREE_LIVE.csv'
gnn_tree_live = pd.read_csv(workdir+'\\'+CSV, sep = ',', quotechar = '"', index_col = 'FCID')
print('Done.')
print(str(len(gnn_tree_live)) + " live trees loaded.")
print(str(len(pd.unique(gnn_tree_live.index))) + " FCIDs loaded.")

In [ ]:
# load the FIA cross reference for wood specific gravity by species
CSV = 'PNWFIADB_REF_SPECIES.csv'
FIA_gravs = pd.read_csv(workdir+'\\'+CSV, sep = ',', quotechar = '"', index_col = 'SPECIES_SYMBOL')

# convert FIA_gravs lookup table to a dictionary
FIA_gravs_dict = FIA_gravs.to_dict(orient = 'index')
print('Done.')

In [ ]:
# # identify the 2" diameter class for each tree
# # takes in a tree DBH and returns the min-max range for 2" diameter class for each tree
# # GNN has diameter in cm, so convert to inches by /2.54

# def minDBH(dbh):
#     return int(np.floor(dbh/2.54/2)*2)

# def maxDBH(dbh):
#     # if a tree has an even DBH (e.g., 2.0), don't let maxDBH = minDBH...
#     if np.ceil(dbh/2)*2 == np.floor(dbh/2.54/2)*2:
#         return int(np.ceil(dbh/2.54/2)*2 +2)
#     else:
#         return int(np.ceil(dbh/2.54/2)*2)

In [ ]:
# # calculate the min and max DBHs for all the trees
# gnn_tree_live["minDBH_in"] = gnn_tree_live.DBH_CM.apply(minDBH)
# gnn_tree_live["maxDBH_in"] = gnn_tree_live.DBH_CM.apply(maxDBH)

In [ ]:
def calc_vols(row, region):
    """
    This function is applied to rows (individual trees) to calculate sawlog_cubic volume and boardfoot volumes.
    It looks up the appropriate volume equation for that tree species in a particular region from the VolumeEq_Dict
    and then calculates the requested volume metric.
    region: 'WOR', 'EOR', 'WWA', 'EWA', or 'CA'
    metric: 'sawlog_cubic', 'boardfoot_16ft', or 'boardfoot_32ft' (other specific metrics may be used, they are identified in ARBCARVolume Equations.py)
    """
    # enforce diameter specifications
    if row.DBH_CM/2.54 < 1:  
        CVTS = 0 # cubic volume of total stem, ground to tip, for DBH >= 1"
        CVT = 0 # cubic volume from 1-foot stump to tip, for DBH >= 1"
    else:
        CVTS = VolumeEq_Dict[row.SPP_SYMBOL][region](row.DBH_CM/2.54,row.HT_M/0.3048, 'CVTS')
        CVT = VolumeEq_Dict[row.SPP_SYMBOL][region](row.DBH_CM/2.54,row.HT_M/0.3048, 'CVT')
    
    if row.DBH_CM/2.54 < 5:
        CV4 = 0 # cubic volume from 1-foot stump to 4" top, for DBH >= 5"
    else:
        CV4 = VolumeEq_Dict[row.SPP_SYMBOL][region](row.DBH_CM/2.54,row.HT_M/0.3048, 'CV4')
    
    # lookup whether the tree is hardwood or softwood
    type = VolumeEq_Dict[row.SPP_SYMBOL]['type']
    
    # enforce softwood diameter limits
    if type == 'SW' and row.DBH_CM/2.54 < 9:
        CV_sawlog = 0 # sawlog cubic volume for softwoods, 1-foot stump to 6" top, for DBH >= 9"
        BF = 0 # Scribner boardfoot volume for softwoods, in 16- or 32-foot log lengths from 1-foot stump to a 6" top, for DBH >= 9"
    elif type == 'SW' and row.DBH_CM/2.54 >= 9 and region in ['EOR', 'EWA', 'CA']: # 16-foot logs for these regions
        CV_sawlog = VolumeEq_Dict[row.SPP_SYMBOL][region](row.DBH_CM/2.54,row.HT_M/0.3048, 'CV6')
        BF = VolumeEq_Dict[row.SPP_SYMBOL][region](row.DBH_CM/2.54,row.HT_M/0.3048, 'SV616')
    elif type == 'SW' and row.DBH_CM/2.54 >= 9 and region in ['WOR', 'WWA']: # 32-foot logs for these regions
        CV_sawlog = VolumeEq_Dict[row.SPP_SYMBOL][region](row.DBH_CM/2.54,row.HT_M/0.3048, 'CV6')
        BF = VolumeEq_Dict[row.SPP_SYMBOL][region](row.DBH_CM/2.54,row.HT_M/0.3048, 'SV632')
        
    # enforce hardwood diameter limits
    if type == 'HW' and row.DBH_CM/2.54 < 11:
        CV_sawlog = 0 # sawlog cubic volume for hardwoods, 1-foot stump to 8" top, for DBH >= 11"
        BF = 0 # Scribner boardfoot volume for hardwoods, in 16-foot log lengths from 1-foot stump to a 8" top, for DBH >= 9"
    elif type == 'HW' and row.DBH_CM/2.54 >= 11:
        CV_sawlog = VolumeEq_Dict[row.SPP_SYMBOL][region](row.DBH_CM/2.54,row.HT_M/0.3048, 'CV8')
        BF = VolumeEq_Dict[row.SPP_SYMBOL][region](row.DBH_CM/2.54,row.HT_M/0.3048, 'SV816')
    
    # report out how far along we are
    if calc_vols.i % 50000 == 0:
        print('{:,}'.format(calc_vols.i)+"..."),
    
    # return all the results
    return CVTS, CVT, CV4, CV_sawlog, BF    

In [ ]:
# calculate cubic and boardfoot volumes for each row, add new columns
for region in ['WOR', 'WWA', 'EOR', 'EWA', 'CA']:
    calc_vols.i = 0 # reset the counter inside calc_vols
    gnn_tree_live[region+'_CVTS_ft3'], gnn_tree_live[region+'_CVT_ft3'], gnn_tree_live[region+'_CV4_ft3'], \
    gnn_tree_live[region+'_CV_sawlog_ft3'], gnn_tree_live[region+'_BF'] = \
    zip(*gnn_tree_live[['SPP_SYMBOL', 'DBH_CM', 'HT_M']].apply(lambda x: calc_vols(x, region), axis = 1))
    print('Done with ' + region +'.')

In [ ]:
# Convert volume metrics to an area basis
# 1 hectare = 2.47105381613712 acres
for region in ['WOR', 'WWA', 'EOR', 'EWA', 'CA']:
    for metric in ['CVT_ft3', 'CVTS_ft3', 'CV4_ft3', 'CV_sawlog_ft3', 'BF']:
        gnn_tree_live_vols[region + "_" + metric + "_ac"] = (gnn_tree_live_vols.TPH_FC * gnn_tree_live_vols[region + "_" + metric] / 2.47105381613712)

print('Done.')

In [ ]:
gnn_tree_live.to_csv('C:\Users\ddiaz\Desktop\GNN_TREE_LIVE_vols_ac.csv')
print('Done.')

In [5]:
CSV = 'GNN_TREE_LIVE_vols_ac.csv'
gnn_tree_live_vols_ac = pd.read_csv(workdir+'\\'+CSV, sep = ',', quotechar = '"', index_col = 'FCID')
print('Done.')
print(str(len(gnn_tree_live_vols_ac)) + " live trees loaded.")
print(str(len(pd.unique(gnn_tree_live_vols_ac.index))) + " FCIDs loaded.")

Done.
2651138 live trees loaded.
53704 FCIDs loaded.


In [ ]:
gnn_tree_live_vols_ac.dtypes

In [ ]:
def get_biomass(spp_symbol, dbh_in, cull_cubic, bio_metric):
    """
    Calculates several biomass metrics (in lbs) using Jenkins equations provided by FIA program
    Can return total above-ground biomass (AG_biomass), gross green stem biomass to 4" top (stem_green_bio_gross),
    net green stem biomass to 4" top adjusted for cull (stem_green_bio_net),
    or bone-dry stem biomass to 4" top (stem_bonedry_bio).
    Looks up the appropriate coefficients for Jenkins equations from FIA crosswalk using NRCS PLANTS symbol for a species.
    Calculates biomass from dbh (provided in inches) and adjusts for cull_cubic (integer from 0-100) recorded in GNN 
    """
    if bio_metric not in ['AG_biomass', 'stem_green_bio_gross', 'stem_green_bio_net', 'stem_bonedry_bio']:
        raise ValueError("bio_metric must be one of 'AG_biomass', 'stem_green_bio_gross', 'stem_green_bio_net', or 'stem_bonedry_bio'.")
        
    # calculate total aboveground biomass (green weight) using Jenkins parameters in FIA crosswalk
    # lookup jenkins_total coefficients
    jenk_total_B1, jenk_total_B2 = None, None
    jenk_total_B1, jenk_total_B2 = FIA_gravs_dict[spp_symbol]['JENKINS_TOTAL_B1'], FIA_gravs_dict[spp_symbol]['JENKINS_TOTAL_B2']
    # = exp(JENKINS_TOTAL_B1 + JENKINS_TOTAL_B2 * ln(DIA*2.54) ) * 2.2046
    AG_biomass = math.exp(jenk_total_B1 + jenk_total_B2 * math.log(dbh_in*2.54)) * 2.2046 # remember that math.log is natural log

    # then calculate stem ratio using parameters in FIA crosswalk
    jenk_stemwoodratio_B1, jenk_stemwoodratio_B2 = None, None
    jenk_stemwoodratio_B1, jenk_stemwoodratio_B2 = FIA_gravs_dict[spp_symbol]['JENKINS_STEM_WOOD_RATIO_B1'], FIA_gravs_dict[spp_symbol]['JENKINS_STEM_WOOD_RATIO_B2']
    # = exp(JENKINS_STEM_WOOD_RATIO_B1 + JENKINS_STEM_WOOD_RATIO_B2 / (DIA*2.54) )
    stem_ratio = math.exp(jenk_stemwoodratio_B1 + jenk_stemwoodratio_B2 / (dbh_in*2.54))
    # then calculate stem biomass using total AG biomass and stem ratio
    # = total_AG_biomass_Jenkins * stem_ratio
    stem_green_bio_gross = AG_biomass * stem_ratio
    # then adjust for cull
    # = VOLUME * (1-CULL_CUBIC/100.0)
    stem_green_bio_net = stem_green_bio_gross * (1 - cull_cubic/100.0)

    # then calculate bone-dry weight
    # lookup specific gravity of green volume
    wood_spgr = None
    wood_spgr = FIA_gravs_dict[spp_symbol]['WOOD_SPGR_GREENVOL_DRYWT']
    # = (soundVOLUME * (WOOD_SPGR_GREENVOL_DRYWT * 62.4) )
    stem_bonedry_bio = stem_green_bio_net * wood_spgr * 62.4
    
    # report out how far along we are
    get_biomass.i += 1
    if get_biomass.i % 50000 == 0:
        print('{:,}'.format(get_biomass.i)+"... "),
    
    # return the requested biomass metric
    return eval(bio_metric)

In [ ]:
get_biomass.i = 0
gnn_tree_live_vols_ac['CV4_bio_tons'] = gnn_tree_live_vols_ac[['SPP_SYMBOL', 'DBH_CM', 'CULL_CUBIC']].apply(lambda row: get_biomass(row.SPP_SYMBOL, row.DBH_CM/2.54, row.CULL_CUBIC, 'stem_bonedry_bio')/2000, axis = 1)
print('Done.')

In [ ]:
# calculate biomass tonnage per acre
gnn_tree_live_vols_ac['CV4_bio_tons_ac'] = gnn_tree_live_vols_ac['CV4_bio_tons'] * gnn_tree_live_vols_ac['TPH_FC'] / 2.47105381613712

In [9]:
# Update the lookup table with new columns that have been added
gnn_tree_live_vols_ac.to_csv('C:\Users\ddiaz\Desktop\GNN_TREE_LIVE_vols_ac.csv')
print('Done.')

Done.
